<a href="https://colab.research.google.com/github/Chovy-Culver/weatherwise-Chufengsheng-Zheng-doe/blob/main/WeatherWise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests

API_KEY = "1da7dea923d784a5dd83bac94cde8d65"
city = "Perth"
url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"

response = requests.get(url)
data = response.json()

# Display main information
print("City:", data['name'])
print("Weather:", data['weather'][0]['description'])
print("Temperature:", data['main']['temp'], "°C")
data = response.json()

# Display main information
print("City:", data['name'])
print("Weather:", data['weather'][0]['description'])
print("Temperature:", data['main']['temp'], "°C")


City: Perth
Weather: clear sky
Temperature: 17.03 °C
City: Perth
Weather: clear sky
Temperature: 17.03 °C
